In [10]:
import pandas as pd
import numpy as np

In [34]:
df = pd.read_csv('data/raw_dataset_dropPrice_str.csv', index_col=[0])

# Data exploration

### Remove duplicate ID

In [35]:
df['Immoweb ID'].value_counts()

9691876    4
9662650    3
9597143    3
8237073    3
9687272    3
          ..
9623733    1
9245768    1
9606238    1
9695276    1
7921699    1
Name: Immoweb ID, Length: 13893, dtype: int64

In [13]:
df = df.drop_duplicates(['Immoweb ID'], keep='last')
df['Immoweb ID'].value_counts()

9729720    1
9678715    1
9697071    1
9557609    1
9678634    1
          ..
9518030    1
9622950    1
9478102    1
9620790    1
7921699    1
Name: Immoweb ID, Length: 13893, dtype: int64

### Remove subtype 

In [14]:
df = df[df['property sub-type'] != "APARTMENT_BLOCK"]
df['property sub-type'].value_counts()

VILLA                   2468
GROUND_FLOOR            1606
DUPLEX                  1580
MIXED_USE_BUILDING      1240
PENTHOUSE               1170
FLAT_STUDIO              873
EXCEPTIONAL_PROPERTY     566
SERVICE_FLAT             500
MANSION                  479
TOWN_HOUSE               383
COUNTRY_COTTAGE          292
LOFT                     271
BUNGALOW                 212
FARMHOUSE                171
TRIPLEX                   99
KOT                       95
CHALET                    94
MANOR_HOUSE               62
CASTLE                    50
Name: property sub-type, dtype: int64

# Data Cleaning

### Building condition

In [15]:
df['Building condition'].value_counts()
df['Building condition'].unique()

array(['Good', nan, 'As new', 'To renovate', 'To be done up',
       'Just renovated', 'To restore'], dtype=object)

In [63]:
building_condition_map = {'As new': 6, 'Just renovated': 5, 'Good': 4, 'To be done up': 3, 'To renovate':2, 'To restore':1}
df = df.applymap(lambda s: building_condition_map.get(s) if s in building_condition_map else s)

In [17]:
df['Building condition'].isnull().sum()

3828

In [18]:
df['Building condition'] = df['Building condition'].fillna(2)
df['Building condition'].isnull().sum()

0

### Kitchen type	


### Bedrooms


### Furnished


In [19]:
df['Furnished'].unique()

array(['Yes', 'No', nan], dtype=object)

In [20]:
df['Furnished'].isnull().sum()

4080

#### fill No to missing value then map to 1,0

In [21]:
df['Furnished'] = df['Furnished'].fillna("No")
df['Furnished'] = df['Furnished'].apply(lambda v: 0 if v == "No" else 1)

#### double check if value is 1, 0 and no missing value

In [22]:
print(df['Furnished'].unique())
print(df['Furnished'].isna().sum())

[1 0]
0


### Terrace surface + Terrace	


### Tenement building	


### Swimming pool	


### How many fireplaces?	


### Garden + Garden surface	


In [23]:
print(df['Garden'].isna().sum())
print(df['Garden'].unique())

11051
[nan 'Yes']


In [24]:
df.loc[df['Garden surface'].isna()]

,Unnamed: 0,Immoweb ID,Property type,property sub-type,Price,Post code,Building condition,Kitchen type,Bedrooms,Furnished,Terrace surface,Tenement building,Number of frontages,Swimming pool,How many fireplaces?,Garden,Garden orientation,Garden surface,Terrace,Surface of the plot
0,0,9729720,APARTMENT,PENTHOUSE,179000.0,1140,4.0,Semi equipped,1.0,1,9.0,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,9729785,APARTMENT,PENTHOUSE,255000.0,8370,4.0,Installed,1.0,1,46.0,No,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,9729784,APARTMENT,PENTHOUSE,255000.0,8370,4.0,Installed,1.0,1,46.0,No,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,9729780,APARTMENT,PENTHOUSE,620000.0,8370,4.0,Installed,3.0,1,70.0,No,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,9727201,APARTMENT,PENTHOUSE,379000.0,1020,4.0,USA hyper equipped,2.0,0,110.0,No,3.0,No,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14138,14536,7770552,APARTMENT,SERVICE_FLAT,132000.0,1730,6.0,NaN,1.0,0,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14139,14537,7770553,APARTMENT,SERVICE_FLAT,177000.0,1730,6.0,NaN,1.0,0,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
14140,14538,7770551,APARTMENT,SERVICE_FLAT,190000.0,1730,6.0,NaN,2.0,0,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN
14141,14539,7921693,APARTMENT,SERVICE_FLAT,1350000.0,5600,2.0,NaN,30.0,0,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Combine ['Garden'] and ['Garden surface] as ['garden_label] and map to 1,0

In [25]:
def categorise(col):  
    if col['Garden'] == "Yes" or col['Garden surface'] > 0:
        return 1
    return 0


df['garden_label'] = df.apply(lambda col: categorise(col), axis=1)

#### double check if value is 1, 0 and no missing value

In [27]:
df['garden_label'].unique()

array([0, 1])

#### drop column Garden and Garden surface

### Garden orientation	


### Surface of the plot




### Final check missing value %




In [49]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Immoweb ID,Immoweb ID,0.000000
Property type,Property type,0.000000
property sub-type,property sub-type,0.000000
Price,Price,0.000000
Post code,Post code,0.000000
Building condition,Building condition,32.128968
Kitchen type,Kitchen type,39.369299
Bedrooms,Bedrooms,16.863466
Furnished,Furnished,34.101676
Terrace surface,Terrace surface,64.703387


# Data Analysis

### Group Post code to region

In [51]:
%pip install pgeocode --quiet

You should consider upgrading via the '/Users/hsinhan/PycharmProjects/challenge_immo_eliza_analysis/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [46]:
import pgeocode
nomi = pgeocode.Nominatim("be")
nomi.query_postal_code("1040")['place_name']

'Etterbeek'

In [30]:
nomi.query_postal_code("5030")['state_name']

'Wallonie'

In [61]:
def get_state(col):  
    region = nomi.query_postal_code(col['Post code'])['state_name']
    if region == "Wallonie":
        region = "Wallonia"
    elif region == "Vlaanderen":
        region = "Flanders"
    else:
        region = "Brussels"
    return region

def get_commune(col):  
    return nomi.query_postal_code(col['Post code'])['place_name']

df['region'] = df.apply(lambda col: get_state(col), axis=1)
df['commune'] = df.apply(lambda col: get_commune(col), axis=1)


In [62]:
df[['commune', 'Post code', 'region']]

,commune,Post code,region
0,Evere,1140,Brussels
1,"Uitkerke, Blankenberge",8370,Flanders
2,"Uitkerke, Blankenberge",8370,Flanders
3,"Uitkerke, Blankenberge",8370,Flanders
4,Laeken,1020,Brussels
...,...,...,...
14536,"Asse, Kobbegem, Bekkerzeel, Mollem",1730,Flanders
14537,"Asse, Kobbegem, Bekkerzeel, Mollem",1730,Flanders
14538,"Asse, Kobbegem, Bekkerzeel, Mollem",1730,Flanders
14539,"Jamiolle, Vodecée, Philippeville, Villers-En-F...",5600,Wallonia
